O objetivo deste trabalho é realizar uma classificação de tipos de choro infantil usando redes neurais convolucionais.

Vamos utilizar o banco de dados "Donate a Cry", disponível no Kaggle: https://www.kaggle.com/datasets/warcoder/infant-cry-audio-corpus/data.

O banco é composto por amostras de choro infantil de 5 tipos:
- Dor
- Gases
- Desconforto
- Fome
- Sono

Essa classificação foi realizada pelos pais/tutores dos bebês, baseados no que cessou o choro. 

Para alcançar nosso objetivo vamos seguir o fluxo:
- Ler os dados
- Transformar o sinal em imagem (espectrogramas)
- Desenvolver uma CNN
- Treinar a CNN
- Testar o desempenho com uma nova amostra de dados

In [1]:
import numpy as np
import os
import pandas as pd
import pathlib as Path
import librosa
import librosa.display
import matplotlib.pyplot as plt


In [2]:
def create_spectrogram(audio_file, image_file):
    # Cria a figura para o espectrograma
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)

    # Carrega o áudio e cria o espectrograma
    y, sr = librosa.load(audio_file)
    ms = librosa.feature.melspectrogram(y=y, sr=sr)
    log_ms = librosa.power_to_db(ms, ref=np.max)
    librosa.display.specshow(log_ms, sr=sr)

    # Salva o espectrograma como imagem
    fig.savefig(image_file)
    plt.close(fig)

def create_pngs_from_wavs(input_path, output_path):
    # Cria a pasta de saída se não existir
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Lista os arquivos na pasta de entrada
    dir = os.listdir(input_path)

    # Processa cada arquivo .wav e gera o espectrograma
    for i, file in enumerate(dir):
        if file.endswith('.wav'):  # Verifica se é um arquivo .wav
            input_file = os.path.join(input_path, file)
            output_file = os.path.join(output_path, file.replace('.wav', '.png'))
            create_spectrogram(input_file, output_file)

In [3]:
pain_input = r"C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\belly_pain"
pain_output = r"C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\output_pain"
hungry_input = r"C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\hungry"
hungry_output = r"C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\output_hungry"

In [4]:
# Chama a função para gerar os espectrogramas
create_pngs_from_wavs(pain_input, pain_output)
create_pngs_from_wavs(hungry_input, hungry_output)


In [5]:
import tensorflow

In [6]:
x = []
y = []

from keras.preprocessing import image # type: ignore

def load_images_from_path(path, label):
    images = [] #lista para armazenar as imagens
    labels = [] #lista para armazenar os rótulos

    for file in os.listdir(path): #itera sobre os arquivos da pasta
        images.append(image.img_to_array(image.load_img(os.path.join(path, file), target_size=(224, 224, 3)))) #carrega, redimensiona e transforma as imagens em arrays
        labels.append((label))

    return images, labels

In [7]:
from pathlib import Path
import shutil

def move_files(source_pattern, destination_path):
    # Cria o diretório de destino se não existir
    destination = Path(destination_path)
    destination.mkdir(parents=True, exist_ok=True)

    # Obtém o diretório de origem a partir do padrão
    source_dir = Path(source_pattern).parent
    file_pattern = Path(source_pattern).name

    # Encontra e move os arquivos que correspondem ao padrão
    for file_path in source_dir.glob(file_pattern):
        shutil.move(str(file_path), destination)
        print(f"Moved {file_path} to {destination}")

# Define seus diretórios e padrões de origem
directories = {
    r'C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\pain_train/': r'C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\output_pain\*.png',
    r'C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\hungry_train/': r'C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\output_hungry\*.png'
}

# Mover os arquivos
for directory, source_pattern in directories.items():
    move_files(source_pattern, directory)


Moved C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\output_pain\549a46d8-9c84-430e-ade8-97eae2bef787-1430130772174-1.7-m-48-bp.png to C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\pain_train
Moved C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\output_pain\643D64AD-B711-469A-AF69-55C0D5D3E30F-1430138495-1.0-m-72-bp.png to C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\pain_train
Moved C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\output_pain\643D64AD-B711-469A-AF69-55C0D5D3E30F-1430138506-1.0-m-72-bp.png to C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\pain_train
Moved C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\output_pain\643D64AD-B711-469A-AF69-55C0D5D3E30F-1430138514-1.0-m-72-bp.png to C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donat

In [8]:
images, labels = load_images_from_path(r'C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\pain_train', 0)

x += images
y += labels



In [9]:
images, labels = load_images_from_path(r'C:\Users\hanna\OneDrive\Documentos\Doutorado\piloto_donateacry\donateacry_corpus\hungry_train', 1)

x += images
y += labels